In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import doubletdetection
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['font.sans-serif'] = "Arial"
import scvelo as scv
scv.logging.print_version()
scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.presenter_view = True  # set max width size for presenter view
scv.settings.set_figure_params('scvelo')  # for beautified visualizationscv.settings.set_figure_params('scvelo') 
sc.settings.set_figure_params(dpi = 50, dpi_save = 600, figsize = (4, 4))

Running scvelo 0.2.4 (python 3.9.7) on 2023-02-18 09:35.


ERROR: XMLRPC request failed [code: -32500]
RuntimeError: PyPI no longer supports 'pip search' (or XML-RPC search). Please use https://pypi.org/search (via a browser) instead. See https://warehouse.pypa.io/api-reference/xml-rpc.html#deprecated-methods for more information.


In [2]:
adata = sc.read('/public/home/zhaox/pigscRNA-seq/intergedbbknn_germsubset1212.h5ad') 
adata

AnnData object with n_obs × n_vars = 11696 × 2698
    obs: 'Sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'Sample_colors', 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [19]:
adata.obs.cell_type

adult_AAACCCAAGATGTAGT-1         Round spermatids
adult_AAACCCAAGCCGCTTG-1         Round spermatids
adult_AAACCCAAGGCTCTAT-1       Diff spermatogonia
adult_AAACCCAAGTAAACAC-1    Elongating spermatids
adult_AAACCCAAGTGGAAGA-1    Elongating spermatids
                                    ...          
neo_TTTCCTCCACTAGTAC-1         Diff spermatogonia
neo_TTTCCTCTCACCCTGT-1         Diff spermatogonia
neo_TTTGGTTGTTCGGCCA-1         Diff spermatogonia
neo_TTTGTTGCAGGCTCTG-1         Diff spermatogonia
neo_TTTGTTGTCGCACGAC-1       Undiff spermatogonia
Name: cell_type, Length: 11696, dtype: category
Categories (5, object): ['Undiff spermatogonia', 'Diff spermatogonia', 'Spermatocytes', 'Round spermatids', 'Elongating spermatids']

In [20]:
interged_germsubsetforscvelo = adata[adata.obs['cell_type'].isin(['Undiff spermatogonia', 'Diff spermatogonia'])].copy()
interged_germsubsetforscvelo

AnnData object with n_obs × n_vars = 4255 × 2698
    obs: 'Sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'Sample_colors', 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [21]:
interged_germsubsetforscvelo.obs.Sample

adult_AAACCCAAGGCTCTAT-1       Adult
adult_AAACCCAGTCCAGGTC-1       Adult
adult_AAACCCAGTTCGGCGT-1       Adult
adult_AAACCCATCGCAGATT-1       Adult
adult_AAACGAAAGGTGCTAG-1       Adult
                              ...   
neo_TTTCCTCCACTAGTAC-1      Neonatal
neo_TTTCCTCTCACCCTGT-1      Neonatal
neo_TTTGGTTGTTCGGCCA-1      Neonatal
neo_TTTGTTGCAGGCTCTG-1      Neonatal
neo_TTTGTTGTCGCACGAC-1      Neonatal
Name: Sample, Length: 4255, dtype: category
Categories (2, object): ['Adult', 'Neonatal']

In [22]:
interged_germsubsetforscvelo = interged_germsubsetforscvelo.raw.to_adata()
interged_germsubsetforscvelo

AnnData object with n_obs × n_vars = 4255 × 19094
    obs: 'Sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'Sample_colors', 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    obsp: 'connectivities', 'distances'

In [23]:
interged_germsubsetforscvelo

AnnData object with n_obs × n_vars = 4255 × 19094
    obs: 'Sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'Sample_colors', 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    obsp: 'connectivities', 'distances'

# 解决报错 KeyError: 'base'

In [24]:
interged_germsubsetforscvelo.uns['log1p']["base"] = None

In [25]:
sc.pp.highly_variable_genes(interged_germsubsetforscvelo, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [26]:
sc.pl.highly_variable_genes(interged_germsubsetforscvelo, save= "intergedgermfindhvgsubset0218.pdf")
interged_germsubsetforscvelo.raw = interged_germsubsetforscvelo
interged_germsubsetforscvelo = interged_germsubsetforscvelo[:, interged_germsubsetforscvelo.var.highly_variable]
sc.pp.regress_out(interged_germsubsetforscvelo, ['total_counts', 'pct_counts_mt', 'S_score', 'G2M_score'])
sc.pp.scale(interged_germsubsetforscvelo, max_value=10)

In [27]:
sc.tl.pca(interged_germsubsetforscvelo, svd_solver='arpack')
sc.pp.neighbors(interged_germsubsetforscvelo, n_neighbors=10, n_pcs=40)
sc.tl.tsne(interged_germsubsetforscvelo)
sc.tl.umap(interged_germsubsetforscvelo)
sc.tl.leiden(interged_germsubsetforscvelo,resolution=0.3)

In [28]:
sc.settings.set_figure_params(dpi = 50, dpi_save = 600, figsize = (4, 4))
sc.pl.umap(interged_germsubsetforscvelo, color='leiden',legend_loc='right margin',save='interged_germsubsetforscveloleiden0217.pdf')
sc.pl.tsne(interged_germsubsetforscvelo, color='leiden',legend_loc='right margin',save='interged_germsubsetforscveloleiden0217.pdf')

In [29]:
sc.settings.set_figure_params(dpi = 50, dpi_save = 600, figsize = (4, 4))
sc.pl.umap(interged_germsubsetforscvelo, color='Sample',legend_loc='right margin',save='intergedsample_germsubsetforscveloleiden0217.pdf')

In [30]:
sc.tl.rank_genes_groups(interged_germsubsetforscvelo, 'leiden', method='wilcoxon')

In [31]:
pd.DataFrame(interged_germsubsetforscvelo.uns['rank_genes_groups']['names']).head(5)
interged_germsubsetforscvelo.uns['rank_genes_groups'].keys()
#dict_keys(['logfoldchanges', 'names', 'params', 'pvals', 'pvals_adj', 'scores'])
result = interged_germsubsetforscvelo.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']}).head(5)
res = pd.DataFrame(    
    {group + '_' + key: result[key][group]    
    for group in groups for key in ['names', 'pvals','logfoldchanges','pvals_adj','scores']})
res.to_csv("./diff_rank_genes_groups_interged_germsubsetforscveloleiden0218.csv")

In [32]:
cell_cycle_genes = [x.strip() for x in open('/public/home/zhaox/pigscRNA-seq/cell_cycle_genes (remove).txt')]
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]

In [33]:
ax = sc.pl.heatmap(interged_germsubsetforscvelo, s_genes, groupby='leiden', vmin=-2, vmax=2, cmap='RdBu_r', dendrogram=False, show_gene_labels=False,swap_axes=True, figsize=(11,4),save='g1scellcycle_intergedgermsubsetforscvelo0218.pdf')
ax = sc.pl.heatmap(interged_germsubsetforscvelo, g2m_genes, groupby='leiden', vmin=-2, vmax=2, cmap='RdBu_r', dendrogram=False, show_gene_labels=False,swap_axes=True, figsize=(11,4),save='g2mcellcycle_intergedgermsubsetforscvelo0218.pdf')

In [34]:
results_file = './interged_germsubsetforscvelo_0218unannotation.h5ad'
interged_germsubsetforscvelo.write(results_file)

In [61]:
interged_germsubsetforscvelo.obs.leiden

adult_AAACCCAAGGCTCTAT-1    3
adult_AAACCCAGTCCAGGTC-1    3
adult_AAACCCAGTTCGGCGT-1    3
adult_AAACCCATCGCAGATT-1    1
adult_AAACGAAAGGTGCTAG-1    1
                           ..
neo_TTTCCTCCACTAGTAC-1      8
neo_TTTCCTCTCACCCTGT-1      8
neo_TTTGGTTGTTCGGCCA-1      4
neo_TTTGTTGCAGGCTCTG-1      8
neo_TTTGTTGTCGCACGAC-1      6
Name: leiden, Length: 4255, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

In [62]:
marker_genes_dict = {
    'SSC': ['UCHL1', 'BNC2', 'UTF1','GFRA1','ETV5','FGFR3'],
    'Diffing SPG': ['KIT', 'MKI67','SOHLH1','DMRT1'],
    'Diffed SPG': ['STRA8'],
    'Meiotic genes':['SYCP3','SYCP2']
}

In [59]:
sc.tl.dendrogram(interged_germsubsetforscvelo,groupby="leiden")
sc.pl.dotplot(interged_germsubsetforscvelo, marker_genes_dict, 'leiden', standard_scale='var',dendrogram=True,save="intergedgerm_annotation0218.pdf")

categories: 0, 1, 2, etc.
var_group_labels: SSC, Diffing SPG, Diffed SPG, etc.


In [63]:
cluster2annotation = {
     '0': 'Diff SPG',
     '1': 'Early diff SPG1',
     '2': 'Early diff SPG(KIT)',
     '3': 'Early diff SPG2',
     '4': 'SSC1',
     '5': 'Early diff SPG3',
     '6': 'SSC3',
     '7': 'SSC4',
     '8': 'SSC2',
     '9': 'Early diff SPG(KIT)',
}
interged_germsubsetforscvelo.obs['cell_type'] = interged_germsubsetforscvelo.obs['leiden'].map(cluster2annotation).astype('category')

In [64]:
interged_germsubsetforscvelo.obs['cell_type'].unique().tolist()

['Early diff SPG2',
 'Early diff SPG1',
 'Early diff SPG(KIT)',
 'SSC4',
 'Diff SPG',
 'Early diff SPG3',
 'SSC3',
 'SSC1',
 'SSC2']

In [66]:
original_order = ['Early diff SPG2',
 'Early diff SPG1',
 'Early diff SPG(KIT)',
 'SSC4',
 'Diff SPG',
 'Early diff SPG3',
 'SSC3',
 'SSC1',
 'SSC2']
interged_germsubsetforscvelo.obs['cell_type'] = interged_germsubsetforscvelo.obs['cell_type'].cat.set_categories(['SSC1', 'SSC2','SSC3', 'SSC4','Early diff SPG1','Early diff SPG2','Early diff SPG3','Early diff SPG(KIT)','Diff SPG'])

In [67]:
sc.settings.set_figure_params(dpi = 50, dpi_save = 600, figsize = (4, 4))
sc.pl.umap(interged_germsubsetforscvelo, color='cell_type', frameon=True, legend_loc= "right margin", legend_fontsize=10, legend_fontweight="normal", legend_fontoutline=2, save="interged_germcelltyperight0219.pdf")

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/scanpy/_settings.py:447: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)


In [68]:
cell_cycle_genes = [x.strip() for x in open('/public/home/zhaox/pigscRNA-seq/cell_cycle_genes (remove).txt')]
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]

In [69]:
ax = sc.pl.heatmap(interged_germsubsetforscvelo, s_genes, groupby='cell_type', vmin=-2, vmax=2, cmap='RdBu_r', dendrogram=False, show_gene_labels=False,swap_axes=True, figsize=(11,3),save='g1scellcycle_intergedgerm0219.pdf')
ax = sc.pl.heatmap(interged_germsubsetforscvelo, g2m_genes, groupby='cell_type', vmin=-2, vmax=2, cmap='RdBu_r', dendrogram=False, show_gene_labels=False,swap_axes=True, figsize=(11,4),save='g2mcellcycle_intergedgerm0219.pdf')

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/scanpy/plotting/_anndata.py:2511: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.
  pl.colorbar(mappable, cax=heatmap_cbar_ax)


/public/home/zhaox/anaconda3/lib/python3.9/site-packages/scanpy/plotting/_anndata.py:2511: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.
  pl.colorbar(mappable, cax=heatmap_cbar_ax)


In [70]:
sc.tl.rank_genes_groups(interged_germsubsetforscvelo, 'cell_type', method='wilcoxon')

In [71]:
pd.DataFrame(interged_germsubsetforscvelo.uns['rank_genes_groups']['names']).head(5)
interged_germsubsetforscvelo.uns['rank_genes_groups'].keys()
#dict_keys(['logfoldchanges', 'names', 'params', 'pvals', 'pvals_adj', 'scores'])
result = interged_germsubsetforscvelo.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']}).head(5)
res = pd.DataFrame(    
    {group + '_' + key: result[key][group]    
    for group in groups for key in ['names', 'pvals','logfoldchanges','pvals_adj','scores']})
res.to_csv("./diff_rank_genes_groups_intergedgerm_celltype0219.csv")

In [72]:
sc.pl.rank_genes_groups_heatmap(interged_germsubsetforscvelo, use_raw=True,vmin=-3, vmax=3, min_logfoldchange=1,groupby='cell_type',n_genes=50, cmap='RdBu_r', show=False,dendrogram=False, save = "intergedgerm_useraw0219.pdf")

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/scanpy/plotting/_anndata.py:2511: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.
  pl.colorbar(mappable, cax=heatmap_cbar_ax)


{'heatmap_ax': <AxesSubplot: >,
 'groupby_ax': <AxesSubplot: ylabel='cell_type'>,
 'gene_groups_ax': <AxesSubplot: >}

In [73]:
marker_genes = ['UCHL1', 'BNC2', 'UTF1','GFRA1','ETV5','KIT', 'MKI67','SOHLH1','DMRT1','STRA8']
sc.pl.umap(interged_germsubsetforscvelo, color = marker_genes, ncols=5, save="intergedgerm_markers0219.pdf")

In [74]:
interged_germsubsetforscvelo

AnnData object with n_obs × n_vars = 4255 × 4458
    obs: 'Sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'Sample_colors', 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [75]:
results_file = './intergedgerm_annotation0219.h5ad'
interged_germsubsetforscvelo.write(results_file)

In [76]:
interged_germsubsetforcellrank = interged_germsubsetforscvelo.raw.to_adata()
interged_germsubsetforcellrank

AnnData object with n_obs × n_vars = 4255 × 19094
    obs: 'Sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'Sample_colors', 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    obsp: 'connectivities', 'distances'

In [77]:
interged_germsubsetforcellrank.obs.Sample

adult_AAACCCAAGGCTCTAT-1       Adult
adult_AAACCCAGTCCAGGTC-1       Adult
adult_AAACCCAGTTCGGCGT-1       Adult
adult_AAACCCATCGCAGATT-1       Adult
adult_AAACGAAAGGTGCTAG-1       Adult
                              ...   
neo_TTTCCTCCACTAGTAC-1      Neonatal
neo_TTTCCTCTCACCCTGT-1      Neonatal
neo_TTTGGTTGTTCGGCCA-1      Neonatal
neo_TTTGTTGCAGGCTCTG-1      Neonatal
neo_TTTGTTGTCGCACGAC-1      Neonatal
Name: Sample, Length: 4255, dtype: category
Categories (2, object): ['Adult', 'Neonatal']

In [78]:
adata_adult = interged_germsubsetforcellrank[interged_germsubsetforcellrank.obs['Sample'].isin(['Adult'])].copy()
adata_adult

AnnData object with n_obs × n_vars = 3436 × 19094
    obs: 'Sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'Sample_colors', 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    obsp: 'connectivities', 'distances'

In [79]:
adata_neo = interged_germsubsetforcellrank[interged_germsubsetforcellrank.obs['Sample'].isin(['Neonatal'])].copy()
adata_neo

AnnData object with n_obs × n_vars = 819 × 19094
    obs: 'Sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'Sample_colors', 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    obsp: 'connectivities', 'distances'

In [80]:
adataadult_loom = scv.read('/public/home/zhaox/pigscRNA-seq/Adult/Adult.loom', cache=True)
scv.pl.proportions(adataadult_loom,save="./figures/intergedadultproportions0218forscvelo.pdf")
adata_adult = scv.utils.merge(adata_adult, adataadult_loom)
scv.pl.proportions(
    adata_adult,
    groupby='cell_type',
    layers=None,
    highlight='unspliced',
    add_labels_pie=True,
    add_labels_bar=True,
    fontsize=8,
    figsize=(15, 2),
    dpi=600,
    use_raw=True,
    show=True,
    save="./figures/intergedadultgermcellforscvelospliced0218.pdf",
)

saving figure to file ./figures/intergedadultproportions0218forscvelo.pdf
saving figure to file ./figures/intergedadultgermcellforscvelospliced0218.pdf


In [81]:
adataneo_loom = scv.read('/public/home/zhaox/pigscRNA-seq/P5/P5.loom', cache=True)
scv.pl.proportions(adataneo_loom,save="./figures/intergedneoproportions0218forscvelo.pdf")
adata_neo = scv.utils.merge(adata_neo, adataneo_loom)
scv.pl.proportions(
    adata_neo,
    groupby='cell_type',
    layers=None,
    highlight='unspliced',
    add_labels_pie=True,
    add_labels_bar=True,
    fontsize=8,
    figsize=(15, 2),
    dpi=600,
    use_raw=True,
    show=True,
    save="./figures/intergedneogermcellforscvelospliced0218.pdf",
)

saving figure to file ./figures/intergedneoproportions0218forscvelo.pdf
saving figure to file ./figures/intergedneogermcellforscvelospliced0218.pdf


In [82]:
import anndata as ad
adatas = [adata_adult,adata_neo]
adata = ad.concat(adatas,merge="same")
adata

AnnData object with n_obs × n_vars = 4255 × 19094
    obs: 'Sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'

In [83]:
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

Filtered out 6309 genes that are detected 20 counts (shared).
Normalized count data: spliced, unspliced.
Extracted 2000 highly variable genes.
computing neighbors
    finished (0:00:00) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


In [84]:
scv.tl.velocity(adata)

computing velocities
    finished (0:00:02) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)


In [85]:
scv.tl.velocity_graph(adata)

computing velocity graph (using 1/24 cores)


  0%|          | 0/4255 [00:00<?, ?cells/s]

    finished (0:00:17) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)


In [ ]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color='leiden',save='./figures/germcellforscvelo_embeddingstream1224.pdf')

In [86]:
scv.tl.recover_dynamics(adata, n_jobs=8)

recovering dynamics (using 8/24 cores)


  0%|          | 0/1177 [00:00<?, ?gene/s]

    finished (0:01:54) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)


In [87]:
scv.pl.velocity_embedding_stream(
    adata, basis="umap",color='cell_type', legend_fontsize=12, title="", smooth=0.8, min_mass=4,save='./intergedgermforcellrank_embeddingstream0218.pdf'
)

computing velocity embedding
    finished (0:00:01) --> added
    'velocity_umap', embedded velocity vectors (adata.obsm)
saving figure to file ./intergedgermforcellrank_embeddingstream0218.pdf


In [52]:
import scvelo as scv
import scanpy as sc
import cellrank as cr
import numpy as np
scv.settings.verbosity = 3
scv.settings.set_figure_params("scvelo")
cr.settings.verbosity = 2
import warnings
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=DeprecationWarning)

In [88]:
cr.tl.terminal_states(adata, cluster_key="cell_type", weight_connectivities=0.2)

Accessing `adata.obsp['T_fwd']`


/tmp/ipykernel_48715/637574554.py:1: DeprecationWarning: `cellrank.tl.terminal_states` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  cr.tl.terminal_states(adata, cluster_key="cell_type", weight_connectivities=0.2)
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/cellrank/tl/_init_term_states.py:156: DeprecationWarning: `cellrank.tl.transition_matrix` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  kernel = transition_matrix(


Computing transition matrix based on logits using `'deterministic'` mode
Estimating `softmax_scale` using `'deterministic'` mode


  0%|          | 0/4255 [00:00<?, ?cell/s]

Setting `softmax_scale=13.3672`


  0%|          | 0/4255 [00:00<?, ?cell/s]

    Finish (0:00:14)
Using a connectivity kernel with weight `0.2`
Computing transition matrix based on `adata.obsp['connectivities']`
    Finish (0:00:00)
Computing eigendecomposition of the transition matrix
Adding `adata.uns['eigendecomposition_fwd']`
       `.eigendecomposition`
    Finish (0:00:00)
Computing Schur decomposition
When computing macrostates, choose a number of states NOT in `[3]`
Adding `adata.uns['eigendecomposition_fwd']`
       `.schur_vectors`
       `.schur_matrix`
       `.eigendecomposition`
    Finish (0:00:19)
Computing `7` macrostates
Adding `.macrostates`
       `.macrostates_memberships`
       `.coarse_T`
       `.coarse_initial_distribution
       `.coarse_stationary_distribution`
       `.schur_vectors`
       `.schur_matrix`
       `.eigendecomposition`
    Finish (0:00:04)
Adding `adata.obs['terminal_states']`
       `adata.obs['terminal_states_probs']`
       `.terminal_states`
       `.terminal_states_probabilities`
       `.terminal_states_members

In [89]:
cr.pl.terminal_states(adata,color='cell_type')
plt.savefig('./intergedgermforcellrank_terminal_states0218.pdf')

In [90]:
cr.tl.initial_states(adata, cluster_key="cell_type")
cr.pl.initial_states(adata, discrete=True)
plt.savefig('./intergedgermforcellrank_initial_states0218.pdf')

Accessing `adata.obsp['T_bwd']`


/tmp/ipykernel_48715/4092284627.py:1: DeprecationWarning: `cellrank.tl.initial_states` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  cr.tl.initial_states(adata, cluster_key="cell_type")
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/cellrank/tl/_init_term_states.py:156: DeprecationWarning: `cellrank.tl.transition_matrix` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  kernel = transition_matrix(


Computing transition matrix based on logits using `'deterministic'` mode
Estimating `softmax_scale` using `'deterministic'` mode


  0%|          | 0/4255 [00:00<?, ?cell/s]

Setting `softmax_scale=13.3672`


  0%|          | 0/4255 [00:00<?, ?cell/s]

    Finish (0:00:21)
Using a connectivity kernel with weight `0.2`
Computing transition matrix based on `adata.obsp['connectivities']`
    Finish (0:00:00)
Computing eigendecomposition of the transition matrix
Adding `adata.uns['eigendecomposition_bwd']`
       `.eigendecomposition`
    Finish (0:00:00)
Computing Schur decomposition
Adding `adata.uns['eigendecomposition_bwd']`
       `.schur_vectors`
       `.schur_matrix`
       `.eigendecomposition`
    Finish (0:00:19)
Computing `2` macrostates
Adding `.macrostates`
       `.macrostates_memberships`
       `.coarse_T`
       `.coarse_initial_distribution
       `.coarse_stationary_distribution`
       `.schur_vectors`
       `.schur_matrix`
       `.eigendecomposition`
    Finish (0:00:01)
Adding `adata.obs['initial_states']`
       `adata.obs['initial_states_probs']`
       `.terminal_states`
       `.terminal_states_probabilities`
       `.terminal_states_memberships
    Finish`


In [91]:
cr.tl.lineages(adata)
cr.pl.lineages(adata, same_plot=False)
plt.savefig('./intergedgermforcellrank_fatemaps0219.pdf')

Computing absorption probabilities
Defaulting to `'gmres'` solver.


/tmp/ipykernel_48715/3398442375.py:1: DeprecationWarning: `cellrank.tl.lineages` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  cr.tl.lineages(adata)


  0%|          | 0/7 [00:00<?, ?/s]

Adding `adata.obsm['to_terminal_states']`
       `.absorption_probabilities`
    Finish (0:00:01)


/public/home/zhaox/anaconda3/lib/python3.9/site-packages/scvelo/plotting/utils.py:866: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.
  cb = pl.colorbar(smp, orientation=orientation, cax=cax)
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/scvelo/plotting/utils.py:869: MatplotlibDeprecationWarning: The draw_all function was deprecated in Matplotlib 3.6 and will be removed two minor releases later. Use fig.draw_without_rendering() instead.
  cb.draw_all()


In [92]:
scv.tl.recover_latent_time(
    adata, root_key="initial_states_probs", end_key="terminal_states_probs"
)

computing latent time using initial_states_probs, terminal_states_probs as prior
    finished (0:00:03) --> added 
    'latent_time', shared time (adata.obs)


In [93]:
scv.tl.paga(
    adata,
    groups="cell_type",
    root_key="initial_states_probs",
    end_key="terminal_states_probs",
    use_time_prior="velocity_pseudotime",
)

running PAGA using priors: ['velocity_pseudotime', 'initial_states_probs', 'terminal_states_probs']
    finished (0:00:04) --> added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns)
    'paga/transitions_confidence', velocity transitions (adata.uns)


In [94]:
cr.pl.cluster_fates(adata, mode="bar",cluster_key="cell_type")
plt.savefig('./intergedgermforcellrank_barplot_clusterfates0219.pdf')

In [95]:
sc.settings.set_figure_params(dpi = 50, dpi_save = 600, figsize = (12, 5))
cr.pl.cluster_fates(adata, mode="heatmap",cluster_key="cell_type")
plt.savefig('./intergedgermforcellrank_heatmap_clusterfates0219.pdf')

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/scanpy/_settings.py:447: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/seaborn/matrix.py:302: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.
  mesh = ax.pcolormesh(self.plot_data, cmap=self.cmap, **kws)


In [96]:
cr.tl.lineage_drivers(adata)

/tmp/ipykernel_48715/1721556810.py:1: DeprecationWarning: `cellrank.tl.lineage_drivers` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  cr.tl.lineage_drivers(adata)


Adding `adata.varm['terminal_lineage_drivers']`
       `.lineage_drivers`
    Finish (0:00:00)


,SSC2_corr,SSC2_pval,SSC2_qval,SSC2_ci_low,SSC2_ci_high,SSC3_corr,SSC3_pval,SSC3_qval,SSC3_ci_low,SSC3_ci_high,...,Diff SPG_corr,Diff SPG_pval,Diff SPG_qval,Diff SPG_ci_low,Diff SPG_ci_high,Early diff SPG(KIT)_corr,Early diff SPG(KIT)_pval,Early diff SPG(KIT)_qval,Early diff SPG(KIT)_ci_low,Early diff SPG(KIT)_ci_high
COL9A2,0.948912,0.000000e+00,0.000000e+00,0.945832,0.951821,-0.127200,7.459472e-17,2.356863e-16,-0.156649,-0.097524,...,-0.188882,1.141189e-35,3.411628e-35,-0.217695,-0.159741,-0.066204,1.537022e-05,3.525279e-05,-0.096061,-0.036228
AMH,0.931185,0.000000e+00,0.000000e+00,0.927077,0.935070,-0.117250,1.579905e-14,4.626370e-14,-0.146781,-0.087510,...,-0.196557,1.469647e-38,4.756138e-38,-0.225275,-0.167498,-0.069476,5.688656e-06,1.404606e-05,-0.099317,-0.039510
INHA,0.926861,0.000000e+00,0.000000e+00,0.922505,0.930981,-0.116931,1.862350e-14,5.429591e-14,-0.146465,-0.087189,...,-0.205366,4.944787e-42,1.811277e-41,-0.233971,-0.176406,-0.072745,2.015419e-06,5.221293e-06,-0.102569,-0.042790
SERPINA5,0.922965,0.000000e+00,0.000000e+00,0.918387,0.927296,-0.092717,1.333993e-09,3.098705e-09,-0.122424,-0.062844,...,-0.207501,6.717126e-43,2.506390e-42,-0.236077,-0.178566,-0.072416,2.241202e-06,5.783747e-06,-0.102242,-0.042460
ASS1,0.905477,0.000000e+00,0.000000e+00,0.899914,0.910746,-0.113757,9.333310e-14,2.647748e-13,-0.143315,-0.083995,...,-0.165991,8.760546e-28,2.083364e-27,-0.195067,-0.136624,-0.067239,1.127770e-05,2.656702e-05,-0.097091,-0.037266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LOC110257929,-0.253520,4.512187e-64,3.609750e-63,-0.281424,-0.225187,-0.019843,1.956435e-01,2.278900e-01,-0.049861,0.010212,...,0.525935,0.000000e+00,0.000000e+00,0.503849,0.547334,-0.168854,1.036700e-28,1.312279e-27,-0.197899,-0.139514
SMC1B,-0.255863,2.787315e-65,2.266109e-64,-0.283730,-0.227564,0.036464,1.737090e-02,2.360176e-02,0.006422,0.066439,...,0.577745,0.000000e+00,0.000000e+00,0.557373,0.597422,-0.174381,1.509885e-30,2.068336e-29,-0.203364,-0.145093
DAZL,-0.259650,2.906072e-67,2.411678e-66,-0.287455,-0.231407,-0.054468,3.776804e-04,6.141145e-04,-0.084378,-0.024459,...,0.620972,0.000000e+00,0.000000e+00,0.602160,0.639096,-0.166160,7.733250e-28,9.606522e-27,-0.195234,-0.136795
LOC100157935,-0.301237,2.290164e-91,2.336902e-90,-0.328313,-0.273666,0.131636,5.938654e-18,1.947100e-17,0.101991,0.161047,...,0.495703,3.310234e-275,1.839019e-273,0.472696,0.518035,-0.191688,1.036485e-36,1.787043e-35,-0.220466,-0.162576
